In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 4, ndecoderlayers = 2, dropout = 0.1):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters(), lr = 0.00001, betas = (0.9, 0.999))


class environment:    
    def reset(self, npoints, batchsize, nsamples=1):
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints, 2], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints, 2)
        )
        
        self.distance_matrix = (self.points.unsqueeze(1) - self.points.unsqueeze(2)).square().sum(-1).sqrt() # [batchsize * nsamples, npoints, npoints]
        
        self.previous_point = None
        
        self.points_mask = torch.zeros(
                    [self.batchsize, npoints], dtype=torch.bool, device=device
                )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert list(point_index.size()) == [self.batchsize]
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        if self.previous_point != None:
            self.cost += self.distance_matrix.gather(2, self.previous_point.unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, point_index.unsqueeze(1)).squeeze(1)
        
        self.previous_point = point_index
        self.points_mask.scatter_(1, point_index.unsqueeze(1), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
        
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    
    def laststep(self):
        
        assert self.points_sequence.size(1) == self.npoints
        
        self.cost += self.distance_matrix.gather(2, self.points_sequence[:, 0].unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, self.points_sequence[:, -1].unsqueeze(1)).squeeze(1)
    

env = environment()


def evaluate(epochs = 10, npoints = 10, batchsize = 100, nsamples = 2, negative_cutoff = 1):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item())


def train(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        
        NN.eval()
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.15
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 2
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 40) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
   




In [ ]:
train(300000, 100, 100, 8, 1)

43.91679000854492 -356.9982604980469 -363.49395751953125 -364.3341369628906 -362.68536376953125 -362.6854248046875
44.43418884277344 -356.991943359375 -363.50238037109375 -364.933349609375 -363.55645751953125 -363.5564880371094
43.911277770996094 -357.0184326171875 -363.4757995605469 -364.1596984863281 -363.54937744140625 -363.54937744140625
43.983909606933594 -357.021484375 -363.4716491699219 -361.81024169921875 -364.9255065917969 -364.9254150390625
44.37738037109375 -357.027099609375 -363.5189208984375 -364.0438537597656 -363.7513427734375 -363.75128173828125
43.880252838134766 -357.0880432128906 -363.5133361816406 -363.5831298828125 -363.11700439453125 -363.11700439453125
44.23005676269531 -357.1310119628906 -363.4819030761719 -363.6252746582031 -362.9953308105469 -362.995361328125
44.333953857421875 -357.0980529785156 -363.5323486328125 -364.00445556640625 -363.4527282714844 -363.4526672363281
44.30200958251953 -357.08404541015625 -363.5310974121094 -363.55242919921875 -363.2228393

41.389930725097656 -356.0584716796875 -363.4086608886719 -362.2080383300781 -364.1891784667969 -364.1891174316406
41.19275665283203 -356.16943359375 -363.3831787109375 -363.820556640625 -362.9124755859375 -362.9124450683594
41.10301971435547 -355.9734191894531 -363.46270751953125 -363.4998779296875 -363.87750244140625 -363.87744140625
41.21272277832031 -356.0728759765625 -363.4636535644531 -363.27117919921875 -363.2428283691406 -363.24273681640625
41.135093688964844 -356.1863098144531 -363.396728515625 -363.1685791015625 -362.88287353515625 -362.88299560546875
40.606689453125 -356.11944580078125 -363.4012145996094 -363.0736083984375 -362.91998291015625 -362.9200439453125
41.61223220825195 -356.1296081542969 -363.42767333984375 -362.26141357421875 -364.39703369140625 -364.39715576171875
41.11895751953125 -356.1129150390625 -363.3798522949219 -363.14215087890625 -363.052490234375 -363.0525207519531
41.05095291137695 -356.2413330078125 -363.4352722167969 -363.76922607421875 -363.142944335

41.73571014404297 -356.6750793457031 -363.51904296875 -362.8706359863281 -363.3047180175781 -363.3047790527344
41.877410888671875 -356.7242736816406 -363.4726257324219 -363.1018371582031 -363.2392578125 -363.23919677734375
41.63629913330078 -356.6522521972656 -363.4732360839844 -362.4239501953125 -364.30145263671875 -364.301513671875
42.088523864746094 -356.6520080566406 -363.4525451660156 -363.01763916015625 -362.90911865234375 -362.9091796875
42.01629638671875 -356.6304626464844 -363.50604248046875 -364.1051940917969 -364.03662109375 -364.0365295410156
41.96003723144531 -356.681640625 -363.4832458496094 -362.80865478515625 -362.2636413574219 -362.2635192871094
42.043304443359375 -356.6813049316406 -363.485107421875 -362.96258544921875 -363.78961181640625 -363.7896423339844
41.97349548339844 -356.69927978515625 -363.47662353515625 -363.76690673828125 -363.0955810546875 -363.0955505371094
42.109519958496094 -356.6729736328125 -363.49652099609375 -364.25750732421875 -364.1614685058594 -

43.7839241027832 -356.6477355957031 -363.4567565917969 -362.79803466796875 -365.526611328125 -365.52655029296875
43.82338333129883 -356.5576171875 -363.4896545410156 -363.0610046386719 -363.18463134765625 -363.18463134765625
43.59504318237305 -356.4844055175781 -363.5074462890625 -363.3699645996094 -363.03277587890625 -363.03271484375
43.78345489501953 -356.5135192871094 -363.46051025390625 -363.0525817871094 -363.1873779296875 -363.1873779296875
43.535057067871094 -356.46832275390625 -363.4812927246094 -363.19293212890625 -363.1182861328125 -363.118408203125
43.5562629699707 -356.49273681640625 -363.4734191894531 -364.4520263671875 -363.8736572265625 -363.8738098144531
43.25908279418945 -356.4901123046875 -363.4864807128906 -364.46380615234375 -362.9691162109375 -362.9690856933594
43.26699447631836 -356.50823974609375 -363.495361328125 -364.6983642578125 -363.70343017578125 -363.7033996582031
43.63627243041992 -356.4627990722656 -363.4692077636719 -364.07330322265625 -363.134765625 -3

43.94880676269531 -356.2913513183594 -363.5208740234375 -364.80560302734375 -362.513427734375 -362.5134582519531
44.42366027832031 -356.27288818359375 -363.468994140625 -363.31842041015625 -362.6838073730469 -362.6838073730469
44.007083892822266 -356.27099609375 -363.44659423828125 -363.533447265625 -363.0228271484375 -363.02288818359375
43.744876861572266 -356.29254150390625 -363.41058349609375 -362.95703125 -362.213623046875 -362.213623046875
44.18031311035156 -356.22357177734375 -363.44757080078125 -364.853271484375 -364.0172119140625 -364.01715087890625
44.04030227661133 -356.2052307128906 -363.4660339355469 -362.7078552246094 -364.141845703125 -364.1419677734375
43.66251754760742 -356.1729736328125 -363.4395751953125 -362.65289306640625 -363.17218017578125 -363.1721496582031
43.80236053466797 -356.246826171875 -363.470458984375 -362.70941162109375 -363.91943359375 -363.9195251464844
43.3674201965332 -356.2424011230469 -363.3939514160156 -364.1566162109375 -363.534423828125 -363.53

...

35.31098937988281 -333.4534606933594 -357.96795654296875 -353.923583984375 -358.9729309082031 -358.9728698730469
35.090354919433594 -333.83197021484375 -358.100341796875 -358.80828857421875 -356.5533447265625 -356.5533142089844
34.79339599609375 -333.5655822753906 -357.8183288574219 -359.8235168457031 -351.1943054199219 -351.1942443847656
34.95490264892578 -333.5827941894531 -357.8550720214844 -358.6236572265625 -352.7884826660156 -352.7884521484375
34.921443939208984 -333.70074462890625 -357.7036437988281 -351.09857177734375 -356.151123046875 -356.15118408203125
35.15049743652344 -334.2513122558594 -358.07098388671875 -354.70635986328125 -360.92779541015625 -360.9277648925781
35.03764724731445 -333.72821044921875 -357.9281311035156 -358.0769348144531 -360.14373779296875 -360.1436767578125
34.81775665283203 -333.660888671875 -357.8883361816406 -360.20452880859375 -354.45050048828125 -354.4504089355469
34.58064651489258 -333.61761474609375 -357.8135681152344 -359.97869873046875 -360.834

35.77641296386719 -335.4342956542969 -358.338623046875 -354.4546813964844 -358.19580078125 -358.19573974609375
35.414512634277344 -335.09600830078125 -358.636962890625 -359.2186584472656 -359.3417053222656 -359.3417053222656
35.42050552368164 -335.32672119140625 -358.5898742675781 -357.86669921875 -361.26446533203125 -361.2644958496094
35.26251220703125 -335.07720947265625 -358.4587707519531 -357.33380126953125 -356.93292236328125 -356.932861328125
35.458534240722656 -335.34515380859375 -358.5215148925781 -357.0035400390625 -362.2372131347656 -362.2372741699219
35.36954116821289 -335.136474609375 -358.64691162109375 -365.8973083496094 -359.921630859375 -359.9216003417969
35.78586959838867 -335.58709716796875 -358.5788269042969 -357.416015625 -360.08172607421875 -360.0816650390625
35.42961883544922 -335.41546630859375 -358.68353271484375 -358.56903076171875 -359.47509765625 -359.47503662109375
34.80385971069336 -335.6171875 -358.3354797363281 -357.428466796875 -357.519775390625 -357.519

35.10190963745117 -336.8049011230469 -359.0411071777344 -357.56121826171875 -362.88336181640625 -362.8834533691406
35.715633392333984 -336.7810974121094 -359.2859191894531 -356.845458984375 -360.119873046875 -360.1197509765625
35.38145065307617 -336.79925537109375 -359.1133728027344 -355.73944091796875 -356.504638671875 -356.504638671875
35.3021240234375 -336.98016357421875 -359.1179504394531 -365.6120910644531 -357.97076416015625 -357.97064208984375
35.15885543823242 -336.74609375 -358.99151611328125 -361.976806640625 -358.2315368652344 -358.2315673828125
35.405330657958984 -336.74249267578125 -359.1317443847656 -351.8702392578125 -355.2628173828125 -355.2627868652344
35.707054138183594 -336.85906982421875 -359.1215515136719 -352.88397216796875 -355.3659362792969 -355.3659973144531
35.48627471923828 -336.7225036621094 -359.22186279296875 -360.73626708984375 -361.8403625488281 -361.84039306640625
35.28650665283203 -336.7330322265625 -359.0054931640625 -354.17779541015625 -359.106719970

35.077449798583984 -334.4035949707031 -358.0834655761719 -352.636474609375 -358.65545654296875 -358.65545654296875
35.60093688964844 -334.9964904785156 -358.4178466796875 -363.96453857421875 -355.9258728027344 -355.92596435546875
34.65663528442383 -334.3351745605469 -358.1729736328125 -356.5379638671875 -361.91827392578125 -361.9182434082031
35.14059829711914 -334.7785949707031 -358.26788330078125 -360.0239562988281 -361.4794616699219 -361.47943115234375
35.48906326293945 -334.5669860839844 -358.44232177734375 -349.936279296875 -359.39459228515625 -359.3946533203125
34.70718765258789 -334.8358459472656 -358.3371887207031 -359.1630859375 -356.1317138671875 -356.13165283203125
35.237430572509766 -335.1240539550781 -358.44708251953125 -354.4839782714844 -358.60321044921875 -358.60333251953125
34.99338150024414 -335.0266418457031 -358.3544006347656 -359.18597412109375 -353.5799560546875 -353.5799560546875
34.81061553955078 -335.3041076660156 -358.63507080078125 -357.01385498046875 -364.901

35.43705368041992 -337.4078063964844 -359.24163818359375 -356.5804748535156 -361.4324645996094 -361.4324645996094
35.73355484008789 -337.548583984375 -359.4456787109375 -356.95074462890625 -357.5921936035156 -357.59210205078125
35.37897491455078 -337.6037292480469 -359.3249206542969 -357.8026123046875 -357.33880615234375 -357.3388671875
35.38026428222656 -337.5539855957031 -359.20263671875 -352.1637878417969 -359.40252685546875 -359.4025573730469
35.604671478271484 -337.6640625 -359.47283935546875 -357.1957092285156 -358.4567565917969 -358.45672607421875
35.25405502319336 -337.0874938964844 -359.2100524902344 -361.29620361328125 -362.5380859375 -362.53802490234375
35.41639709472656 -337.64031982421875 -359.2063903808594 -362.4338684082031 -357.7696533203125 -357.7696838378906
35.45137023925781 -337.3797912597656 -359.1399841308594 -354.02398681640625 -360.336181640625 -360.336181640625
35.36948013305664 -337.3992004394531 -359.3381042480469 -358.063232421875 -356.4009704589844 -356.401

In [ ]:
train(300000, 100, 100, 8, 1)

In [7]:
eval(10, 20, 100)

3.9546682834625244 -0.4706580638885498
3.9321422576904297 -0.4440091550350189
3.9317967891693115 -0.33326441049575806
3.926325798034668 -0.36147579550743103
3.9361000061035156 -0.40054047107696533
3.966549873352051 -0.35143959522247314
3.996304512023926 -0.40718939900398254
3.9640331268310547 -0.36856964230537415
3.977106809616089 -0.4640328586101532
3.972842216491699 -0.45662763714790344
